<a href="https://colab.research.google.com/github/Bigote93/DataInsider/blob/main/DesafioDataInsider_Nalli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#(0) Conexion con Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# (1) Se incorpora los datasets desde Forbes y ciudades


* https://data.world/aroissues/forbes-global-2000-2008-2019
* https://caelum-online-public.s3.amazonaws.com/challenge_data_insider/fortune_global_500.zip
* https://caelum-online-public.s3.amazonaws.com/challenge_data_insider/continentes_y_paises.zip





# (2) Carga de datasets

Se cargara los datos desde la documentacion mediante variables de analisis. Se utilizara metodos de creacion de datasets de la libreria Pandas.

* Importar libreria Pandas
* Se hara una iniciacion de variables de forma dinamica por cada excel
* Se manejara por excepciones la asignacion interna de DataFrames




In [4]:
import pandas as pd

In [142]:
# Cargar los archivos de Excel Forbes de forma dinamica con el metodo global() desde 2000 a 200xx
forbes_datasets = []
forbes_datasets_names = []

for year in range(2008, 2023):
    filename = f"/content/drive/MyDrive/DesafioDataInsiderNalli/aroissues-forbes-global-2000-2008-2019/Forbes Global 2000 - {year} — unified industry.xlsx"

    try:
        data = pd.read_excel(filename)
        variable_name = f"forbesglobal2000_{year}"
        forbes_datasets_names.append(variable_name)
        globals()[variable_name] = data
        print(f"Archivo cargado correctamente: {filename}")
        forbes_datasets.append(globals()[variable_name])

    except Exception as e:
        print(f"Error al cargar el archivo: {filename}")
        print(str(e))

#Las variables seran de la forma: forbesglobal2000_20xx
forbesglobal2000_2018



Archivo cargado correctamente: /content/drive/MyDrive/DesafioDataInsiderNalli/aroissues-forbes-global-2000-2008-2019/Forbes Global 2000 - 2008 — unified industry.xlsx
Archivo cargado correctamente: /content/drive/MyDrive/DesafioDataInsiderNalli/aroissues-forbes-global-2000-2008-2019/Forbes Global 2000 - 2009 — unified industry.xlsx
Archivo cargado correctamente: /content/drive/MyDrive/DesafioDataInsiderNalli/aroissues-forbes-global-2000-2008-2019/Forbes Global 2000 - 2010 — unified industry.xlsx
Archivo cargado correctamente: /content/drive/MyDrive/DesafioDataInsiderNalli/aroissues-forbes-global-2000-2008-2019/Forbes Global 2000 - 2011 — unified industry.xlsx
Archivo cargado correctamente: /content/drive/MyDrive/DesafioDataInsiderNalli/aroissues-forbes-global-2000-2008-2019/Forbes Global 2000 - 2012 — unified industry.xlsx
Archivo cargado correctamente: /content/drive/MyDrive/DesafioDataInsiderNalli/aroissues-forbes-global-2000-2008-2019/Forbes Global 2000 - 2013 — unified industry.xls

,Rank_nr,Company,Industry,Country,Sales,Profits,Assets,Market_Value
0,1,ICBC,Banking,China,165338.0,43669.4,4210927.0,311014.0
1,2,China Construction Bank,Banking,China,143202.0,37203.1,3631583.0,261166.0
2,3,JPMorgan Chase,Banking,United States,118180.0,26496.0,2609785.0,387668.0
3,4,Berkshire Hathaway,Diversified Financials,United States,235165.0,39742.0,702651.0,491888.0
4,5,Agricultural Bank of China,Banking,China,129345.0,29638.7,3439314.0,184125.0
...,...,...,...,...,...,...,...,...
1995,1996,Fabege AB,Construction,Sweden,281.0,873.7,7482.0,3932.0
1996,1996,SEI Investments,Diversified Financials,United States,1572.0,455.5,1894.0,10177.0
1997,1996,Sumec Corporation,Capital Goods,China,11287.0,54.9,6930.0,1205.0
1998,1999,BioMarin Pharmaceutical,Drugs & Biotechnology,United States,1400.0,-144.9,4597.0,15623.0


* Ahora para cargar los datos de fortune_global_20xx debemos usar un webscrapping para extraer datos desde tablas desde documentos html.

In [5]:
from bs4 import BeautifulSoup

In [6]:
# Cargar los archivos de HTML Fortune de forma dinamica con el metodo global() desde 2015 a 2022

ruta_fortune_base = ruta_base = '/content/drive/MyDrive/DesafioDataInsiderNalli/fortune_global_500/fortune_global_'

fortune_datasets = []
fortunes_datasets_names = []

# Cargar las páginas y crear las variables dinamicamente
for year in range(2015, 2023):
    try:
        filename = f"{ruta_base}{year}.html"
        variable_name = f"fortune_global_{year}"
        fortunes_datasets_names.append(variable_name)
        data = pd.read_html(filename)[0]
        globals()[variable_name] = data
        fortune_datasets.append(globals()[variable_name])
        print(f"Archivo cargado correctamente: {filename}")
    except Exception as e:
        print(f"Error al cargar el archivo: {filename}")
        print(str(e))

#Las variables seran de la forma: forbesglobal2000_20xx
fortune_global_2017

Archivo cargado correctamente: /content/drive/MyDrive/DesafioDataInsiderNalli/fortune_global_500/fortune_global_2015.html
Archivo cargado correctamente: /content/drive/MyDrive/DesafioDataInsiderNalli/fortune_global_500/fortune_global_2016.html
Archivo cargado correctamente: /content/drive/MyDrive/DesafioDataInsiderNalli/fortune_global_500/fortune_global_2017.html
Archivo cargado correctamente: /content/drive/MyDrive/DesafioDataInsiderNalli/fortune_global_500/fortune_global_2018.html
Archivo cargado correctamente: /content/drive/MyDrive/DesafioDataInsiderNalli/fortune_global_500/fortune_global_2019.html
Archivo cargado correctamente: /content/drive/MyDrive/DesafioDataInsiderNalli/fortune_global_500/fortune_global_2020.html
Archivo cargado correctamente: /content/drive/MyDrive/DesafioDataInsiderNalli/fortune_global_500/fortune_global_2021.html
Archivo cargado correctamente: /content/drive/MyDrive/DesafioDataInsiderNalli/fortune_global_500/fortune_global_2022.html


,RemoveRank,RemoveName,RemoveRevenues ($M),RemoveRevenue Percent Change,RemoveProfits ($M),RemoveAssets ($M),RemoveProfits Percent Change,RemoveEmployees,RemoveChange in Rank
0,1,Walmart,"$485,873",0.8%,"$13,643","$198,825",-7.2%,2300000,-
1,2,State Grid,"$315,199",-4.4%,"$9,571.3","$489,838",-6.2%,926067,-
2,3,Sinopec Group,"$267,518",-9.1%,"$1,257.9","$310,726",-65%,713288,1
3,4,China National Petroleum,"$262,573",-12.3%,"$1,867.5","$585,619",-73.7%,1512048,-1
4,5,Toyota Motor,"$254,694",7.7%,"$16,899.3","$437,575",-12.3%,364445,3
...,...,...,...,...,...,...,...,...,...
495,496,Teva Pharmaceutical Industries,"$21,903",11.5%,$329,"$92,890",-79.3%,56960,-
496,497,New China Life Insurance,"$21,796",-13.3%,$743.9,"$100,609",-45.6%,54378,-70
497,498,Wm. Morrison Supermarkets,"$21,741",-11.3%,$406.4,"$11,630",20.4%,77210,-61
498,499,TUI,"$21,655",-5.5%,"$1,151.7","$16,247",195.5%,66779,-32



* Ahora cargamos de manera individual los archivos dataset de continente y country debido a su irregularidad de valores.

In [7]:
continente = pd.read_csv('/content/drive/MyDrive/DesafioDataInsiderNalli/continentes_y_paises/continente.csv')
country_code = pd.read_csv('/content/drive/MyDrive/DesafioDataInsiderNalli/continentes_y_paises/country_code.csv')
country_code

,Country,Code
0,Afghanistan,AFG
1,Albania,ALB
2,Algeria,DZA
3,American Samoa,ASM
4,Andorra,AND
...,...,...
217,Virgin Islands,VGB
218,West Bank,WBG
219,Yemen,YEM
220,Zambia,ZMB


# (3) Incluir API polygon.io

* Importamos la api y la instalamos en nuestro cuadernillo

In [8]:
!pip install polygon-api-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 11.3 MB/s eta 0:00:00


In [9]:
from polygon import RESTClient
##from local_settings import polygon as settings

* Cargamos la key de la api

In [10]:
api_key_poligon = 'oL6MR8Fv34GnmLw44bXgp0I62pWmLPm1'

* Ahora inicializamos como cliente utilizando nuestra key de la API

In [11]:
client = RESTClient(api_key=api_key_poligon)
client

#(4)Importaremos los datos historicos .

1. Apple INC (AAPL)
2. Netflix INC (NFLX)
3. Microsoft Corporation (MSFT)
4. Amazon (AMZN)
5. Meta Platforms INC (META)

Luego creamos la funcion que aplicara en cada extraccion de datos

**Utilizamos la guia de : https://github.com/polygon-io/client-python**

In [12]:
#------Amazon------#
ticker = "AMZN"
aggs = []
for a in client.list_aggs(ticker=ticker, multiplier=1, timespan="minute", from_="2022-01-01", to="2022-12-31", limit=50000):
    aggs.append(a)

AMZN = pd.DataFrame(aggs)
AMZN_datafilter = AMZN.loc[:,['open','high','low','close','timestamp']]

In [14]:
#-----Apple INC -----#
ticker = "AAPL"
aggs = []
for a in client.list_aggs(ticker=ticker, multiplier=1, timespan="minute", from_="2022-01-01", to="2022-12-31", limit=50000):
    aggs.append(a)

AAPL = pd.DataFrame(aggs)
AAPL_datafilter = AAPL.loc[:,['open','high','low','close','timestamp']]


In [15]:
#-----Microsoft CorporatioN-----#
ticker = "MSFT"
aggs = []
for a in client.list_aggs(ticker=ticker, multiplier=1, timespan="minute", from_="2022-01-01", to="2022-12-31", limit=50000):
    aggs.append(a)

MSFT = pd.DataFrame(aggs)
MSFT_datafilter = MSFT.loc[:,['open','high','low','close','timestamp']]


In [16]:
#-----Meta Platforms INC-----#
ticker = "META"
aggs = []
for a in client.list_aggs(ticker=ticker, multiplier=1, timespan="minute", from_="2022-01-01", to="2022-12-31", limit=50000):
    aggs.append(a)

META_FB = pd.DataFrame(aggs)
META_FB_datafilter = META_FB.loc[:,['open','high','low','close','timestamp']]


In [18]:
#-----Netflix INC-----#
ticker = "NFLX"
aggs = []
for a in client.list_aggs(ticker=ticker, multiplier=1, timespan="minute", from_="2022-01-01", to="2022-12-31", limit=50000):
    aggs.append(a)

NFLX = pd.DataFrame(aggs)
NFLX_datafilter = NFLX.loc[:,['open','high','low','close','timestamp']]

#(4.1) Metodo solo si se tiene conexion premium

> Esta configuracion de variables globales solo se utiliza si no se sobrepasa la creacion de consultas en polygon.io debido a los limites que tiene la cuenta Free que ofrece.




```
tickers = ["AAPL", "GOOGL", "MSFT", "FB"]

for ticker in tickers:
    aggs = []
    for a in client.list_aggs(ticker=ticker, multiplier=1, timespan="minute", from_="2022-01-01", to="2022-06-13", limit=50000):
        aggs.append(a)
    df = pd.DataFrame(aggs)
    filtered_data = df.loc[:, ['open', 'high', 'low', 'close', 'timestamp']]
    globals()[ticker] = filtered_data
```





* Se cambia la variable timestamp a registro fecha

In [19]:
dataframes_ticker = [
    AMZN_datafilter,
    AAPL_datafilter,
    MSFT_datafilter,
    META_FB_datafilter,
    NFLX_datafilter
]

for dataframes in dataframes_ticker:
  dataframes['timestamp'] = pd.to_datetime(dataframes['timestamp'], unit='ms')

* Ahora transformamos los dataframes a documento CSV y se almacenaran en la carpeta Historicos:

>> /content/drive/MyDrive/DesafioDataInsiderNalli/Historicos



In [20]:
dataframe_name = [
    'AMZN_datafilter',
    'AAPL_datafilter',
    'MSFT_datafilter',
    'META_FB_datafilter',
    'NFLX_datafilter'
]


for dataframes, dataframe_name in zip(dataframes_ticker, dataframe_name):
  ruta_historico_dt = f'/content/drive/MyDrive/DesafioDataInsiderNalli/Historicos/{dataframe_name}.csv'
  dataframes.to_csv(ruta_historico_dt, index=False)



In [21]:
NFLX_datafilter

,open,high,low,close,timestamp
0,604.88,604.88,604.88,604.88,2022-01-03 10:52:00
1,604.70,605.00,604.70,605.00,2022-01-03 13:00:00
2,604.89,604.89,604.89,604.89,2022-01-03 13:19:00
3,604.51,604.51,604.51,604.51,2022-01-03 13:30:00
4,605.00,605.00,605.00,605.00,2022-01-03 13:33:00
...,...,...,...,...,...
130343,295.07,295.07,295.00,295.00,2022-12-31 00:51:00
130344,295.45,295.45,295.20,295.20,2022-12-31 00:54:00
130345,295.44,295.44,295.44,295.44,2022-12-31 00:56:00
130346,295.74,295.74,295.74,295.74,2022-12-31 00:58:00


#(5) Data Wrangling

Debemos hacer lo siguiente a los dataset de Forbes Global 2000:

1. Cambiar el nombre de columnas segun el siguiente detalle

* Company -> Empresa
* Country -> Pais
* Industry -> Industria
* Sales -> Ingresos
* Profits -> Ganancias
* Assets -> Activos
* Market_Value -> Valor de mercado



2.  Generar tres columnas a cada dataframe

* Margen de Rentabilidad: Ganancias/Ingresos
* Rentabilidad de los Activos (ROA): Ganancias/Activos
* Año: Segun el nombre de cada dataset


In [143]:
#Traimos de vuelta el array de los forbes forbes_datasets y hacemos el cambio directo a cada dataframe

for forbes_dataframe in forbes_datasets:
  forbes_dataframe.rename(columns={'Company':'Empresa'}, inplace=True)
  forbes_dataframe.rename(columns={'Country':'Pais'}, inplace=True)
  forbes_dataframe.rename(columns={'Industry':'Industria'}, inplace=True)
  forbes_dataframe.rename(columns={'Sales':'Ingresos'}, inplace=True)
  forbes_dataframe.rename(columns={'Profits':'Ganancias'}, inplace=True)
  forbes_dataframe.rename(columns={'Assets':'Activos'}, inplace=True)
  forbes_dataframe.rename(columns={'Market_Value':'Valor de Mercado'}, inplace=True)


forbesglobal2000_2022


,Rank_nr,Empresa,Industria,Pais,Ingresos,Ganancias,Activos,Valor de Mercado
0,1,Berkshire Hathaway,Diversified Financials,United States,276094,89795.0,958784,741476
1,2,ICBC,Banking,China,208127,54028.3,5518508,214425
2,3,Saudi Arabian Oil Company (Saudi Aramco),Oil & Gas Operations,Saudi Arabia,400376,105363.0,576041,2292077
3,4,JPMorgan Chase,Diversified Financials,United States,124542,42115.0,3954687,374446
4,5,China Construction Bank,Banking,China,202069,46887.3,4746951,181325
...,...,...,...,...,...,...,...,...
1995,1995,Shenzhen Feima International Supply Chain,Business Services & Supplies,China,37,1408.3,166,1136
1996,1997,NMDC,Materials,India,3520,1406.4,5715,6401
1997,1997,Sichuan Changhong Electric,Consumer Durables,China,15716,53.1,12105,1957
1998,1999,Satellite Chemical,Chemicals,China,4413,931.3,7640,9521


In [144]:
#Generamos las dos columnas de Margen de Rentabilida y ROA.
#Luego una columna con los años recorriendo cada elemento del array forbes_datasets_names
#Extraeremos por cadena desde la posicion 17 a la -1

for names_dataset, forbes_dataframe in zip(forbes_datasets_names,forbes_datasets):
  forbes_dataframe['Margen de Rentabilidad'] = forbes_dataframe.Ganancias / forbes_dataframe.Ingresos
  forbes_dataframe['ROA'] = forbes_dataframe.Ganancias / forbes_dataframe.Activos
  forbes_dataframe['Año'] = names_dataset[17:]

forbesglobal2000_2022

,Rank_nr,Empresa,Industria,Pais,Ingresos,Ganancias,Activos,Valor de Mercado,Margen de Rentabilidad,ROA,Año
0,1,Berkshire Hathaway,Diversified Financials,United States,276094,89795.0,958784,741476,0.325233,0.093655,2022
1,2,ICBC,Banking,China,208127,54028.3,5518508,214425,0.259593,0.009790,2022
2,3,Saudi Arabian Oil Company (Saudi Aramco),Oil & Gas Operations,Saudi Arabia,400376,105363.0,576041,2292077,0.263160,0.182909,2022
3,4,JPMorgan Chase,Diversified Financials,United States,124542,42115.0,3954687,374446,0.338159,0.010649,2022
4,5,China Construction Bank,Banking,China,202069,46887.3,4746951,181325,0.232036,0.009877,2022
...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,Shenzhen Feima International Supply Chain,Business Services & Supplies,China,37,1408.3,166,1136,38.062162,8.483735,2022
1996,1997,NMDC,Materials,India,3520,1406.4,5715,6401,0.399545,0.246089,2022
1997,1997,Sichuan Changhong Electric,Consumer Durables,China,15716,53.1,12105,1957,0.003379,0.004387,2022
1998,1999,Satellite Chemical,Chemicals,China,4413,931.3,7640,9521,0.211036,0.121898,2022


#(5.1) A los datasets de Fortune Global 500 le aplicaremos:

1. Cambiar el nombre de las columnas:

* RemoveName -> Empresa
* RemoveEmployees -> Empleados

2. Incorporar una columna para el año

3. Extraer del dataframe solo las columnas Empresa, Empleados y Año

In [49]:
#Incorporamos el cambio de nombre de las columnas de cada dataframe de Fortune
#Luego se incorpora al arreglo auxiliar datasetux que almacenara la extraccion de las columnas de interes del dataframe

datasetux = []

for fortunes_datasets_name, fortune_dataset in zip(fortunes_datasets_names,fortune_datasets):
  fortune_dataset.rename(columns={'RemoveName': 'Empresa', 'RemoveEmployees':'Empleados'}, inplace=True)
  fortune_dataset['Año'] = fortunes_datasets_name[15:]
  fortune_dataset = fortune_dataset[['Empresa','Empleados','Año']]
  datasetux.append(fortune_dataset)



In [50]:
#Se eliminan todas las columnas del dataframe original y se asigna el nuevo dataframe columna por columna

for fortune_dataset, datasetaux in zip(fortune_datasets, datasetux):
  fortune_dataset.drop(fortune_dataset.columns, axis=1, inplace=True)
  fortune_dataset['Empresa'] = datasetaux['Empresa']
  fortune_dataset['Empleados'] = datasetaux['Empleados']
  fortune_dataset['Año'] = datasetaux['Año']

#(6) A los dataset de country_code y continente se le aplicara:

1. Renombrar los nombres de columnas de country de la forma:

* Country -> Nombre de Pais
* Code -> Codigo

2. Renombrar los nombres de la columna de continentes de la forma:

* Continent_Name -> Nombre continente
* Continent_Code -> Codigo Continente
* Three_Letter_Country_Code -> Codigo Pais
* Country_Name -> Nombre de Pais

In [29]:
country_code.rename(columns={'Country':'Nombre de Pais', 'Code':'Codigo'}, inplace=True)

In [30]:
continente.rename(columns={
    'Continent_Name':'Nombre Continente',
    'Continent_Code':'Codigo Continente',
    'Three_Letter_Country_Code': 'Codigo Pais',
    'Country_Name': 'Nombre de Pais'}, inplace=True)

continente

,Nombre Continente,Codigo Continente,Codigo Pais,Nombre de Pais
0,Asia,AS,AFG,"Afghanistan, Islamic Republic of"
1,Europe,EU,ALB,"Albania, Republic of"
2,Antarctica,AN,ATA,Antarctica (the territory South of 60 deg S)
3,Africa,AF,DZA,"Algeria, People's Democratic Republic of"
4,Oceania,OC,ASM,American Samoa
...,...,...,...,...
253,South America,SA,VEN,"Venezuela, Bolivarian Republic of"
254,Oceania,OC,WLF,Wallis and Futuna
255,Oceania,OC,WSM,"Samoa, Independent State of"
256,Asia,AS,YEM,Yemen


#(7) Juncion de tablas y analisis exploratorio

In [145]:
#Hacemos el merge considerando outer para aquellos paises que no tienen empresas o empresas que no tienen pais
forbesglobal2000_2022 = forbesglobal2000_2022.merge(country_code, how = 'outer', left_on ='Pais', right_on='Nombre de Pais')

forbesglobal2000_2022

,Rank_nr,Empresa,Industria,Pais,Ingresos,Ganancias,Activos,Valor de Mercado,Margen de Rentabilidad,ROA,Año,Nombre de Pais,Codigo
0,1,Berkshire Hathaway,Diversified Financials,United States,276094.0,89795.0,958784.0,741476.0,0.325233,0.093655,2022,United States,USA
1,4,JPMorgan Chase,Diversified Financials,United States,124542.0,42115.0,3954687.0,374446.0,0.338159,0.010649,2022,United States,USA
2,6,Amazon,Retailing,United States,469822.0,33364.0,420549.0,1468398.0,0.071014,0.079334,2022,United States,USA
3,7,Apple,Technology Hardware & Equipment,United States,378697.0,100555.0,381191.0,2640316.0,0.265529,0.263792,2022,United States,USA
4,9,Bank of America,Banking,United States,96826.0,30995.0,3238223.0,303102.0,0.320110,0.009572,2022,United States,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2161,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Virgin Islands,VGB
2162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,West Bank,WBG
2163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yemen,YEM
2164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zambia,ZMB


In [146]:
#Hacemos el segundo merge

forbesglobal2000_2022 = forbesglobal2000_2022.merge(continente, how = 'outer', left_on = 'Codigo', right_on='Codigo Pais' )

forbesglobal2000_2022

,Rank_nr,Empresa,Industria,Pais,Ingresos,...,Codigo,Nombre Continente,Codigo Continente,Codigo Pais,Nombre de Pais_y
0,1,Berkshire Hathaway,Diversified Financials,United States,276094.0,...,USA,North America,NaN,USA,United States of America
1,4,JPMorgan Chase,Diversified Financials,United States,124542.0,...,USA,North America,NaN,USA,United States of America
2,6,Amazon,Retailing,United States,469822.0,...,USA,North America,NaN,USA,United States of America
3,7,Apple,Technology Hardware & Equipment,United States,378697.0,...,USA,North America,NaN,USA,United States of America
4,9,Bank of America,Banking,United States,96826.0,...,USA,North America,NaN,USA,United States of America
...,...,...,...,...,...,...,...,...,...,...,...
2232,NaN,NaN,NaN,NaN,NaN,...,NaN,Europe,EU,SJM,Svalbard & Jan Mayen Islands
2233,NaN,NaN,NaN,NaN,NaN,...,NaN,Oceania,OC,TKL,Tokelau
2234,NaN,NaN,NaN,NaN,NaN,...,NaN,North America,NaN,TCA,Turks and Caicos Islands
2235,NaN,NaN,NaN,NaN,NaN,...,NaN,North America,NaN,VIR,United States Virgin Islands


In [148]:
#Generamos la asignacion de los valores NaN a 0 en la columna Rank_nr
forbesglobal2000_2022['Rank_nr'].fillna(-1, inplace = True)

#Verificamos los valores unicos de cada Rank_nr y si existen valores string o boolean
for valores in forbesglobal2000_2022['Rank_nr'].unique():
  if type(valores) == str:
    print(f'Existen valores de tipo string. Estos son: "{valores}"')
  if type(valores) == bool:
    print(f'Existen valores de tipo boolean. Estos son: "{valores}"')

#Cambiamos los valores de Rank_nr de aquellos que son True por 0
forbesglobal2000_2022['Rank_nr'] = forbesglobal2000_2022['Rank_nr'].replace('True', 0)

#Ordenamos de manera ascendente
forbesglobal2000_2022.sort_values('Rank_nr', ascending = True)

Existen valores de tipo string. Estos son: "True"


,Rank_nr,Empresa,Industria,Pais,Ingresos,Ganancias,Activos,Valor de Mercado,Margen de Rentabilidad,ROA,Año,Nombre de Pais_x,Codigo,Nombre Continente,Codigo Continente,Codigo Pais,Nombre de Pais_y
2236,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oceania,OC,WLF,Wallis and Futuna
2105,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Guinea,GIN,Africa,AF,GIN,"Guinea, Republic of"
2104,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Guinea-Bissau,GNB,Africa,AF,GNB,"Guinea-Bissau, Republic of"
2103,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Guernsey,GGY,Europe,EU,GGY,"Guernsey, Bailiwick of"
2102,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Guatemala,GTM,North America,NaN,GTM,"Guatemala, Republic of"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878,1995,Shenzhen Feima International Supply Chain,Business Services & Supplies,China,37.0,1408.3,166.0,1136.0,38.062162,8.483735,2022,China,CHN,Asia,AS,CHN,"China, People's Republic of"
879,1997,Sichuan Changhong Electric,Consumer Durables,China,15716.0,53.1,12105.0,1957.0,0.003379,0.004387,2022,China,CHN,Asia,AS,CHN,"China, People's Republic of"
1587,1997,NMDC,Materials,India,3520.0,1406.4,5715.0,6401.0,0.399545,0.246089,2022,India,IND,Asia,AS,IND,"India, Republic of"
880,1999,Satellite Chemical,Chemicals,China,4413.0,931.3,7640.0,9521.0,0.211036,0.121898,2022,China,CHN,Asia,AS,CHN,"China, People's Republic of"


In [149]:
forbesglobal2000_2022['Rank_nr'].dtype

dtype('int64')

In [150]:
#Eliminamos la columna Pais del dataset que era la columna inicial
forbesglobal2000_2022.drop('Pais', axis = 1, inplace = True)

forbesglobal2000_2022

,Rank_nr,Empresa,Industria,Ingresos,Ganancias,Activos,Valor de Mercado,Margen de Rentabilidad,ROA,Año,Nombre de Pais_x,Codigo,Nombre Continente,Codigo Continente,Codigo Pais,Nombre de Pais_y
0,1,Berkshire Hathaway,Diversified Financials,276094.0,89795.0,958784.0,741476.0,0.325233,0.093655,2022,United States,USA,North America,NaN,USA,United States of America
1,4,JPMorgan Chase,Diversified Financials,124542.0,42115.0,3954687.0,374446.0,0.338159,0.010649,2022,United States,USA,North America,NaN,USA,United States of America
2,6,Amazon,Retailing,469822.0,33364.0,420549.0,1468398.0,0.071014,0.079334,2022,United States,USA,North America,NaN,USA,United States of America
3,7,Apple,Technology Hardware & Equipment,378697.0,100555.0,381191.0,2640316.0,0.265529,0.263792,2022,United States,USA,North America,NaN,USA,United States of America
4,9,Bank of America,Banking,96826.0,30995.0,3238223.0,303102.0,0.320110,0.009572,2022,United States,USA,North America,NaN,USA,United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2232,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Europe,EU,SJM,Svalbard & Jan Mayen Islands
2233,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oceania,OC,TKL,Tokelau
2234,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,North America,NaN,TCA,Turks and Caicos Islands
2235,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,North America,NaN,VIR,United States Virgin Islands


In [151]:
#Eliminamos la columna Codigo Pais,  del dataset que era la columna inicial
forbesglobal2000_2022.drop(['Nombre de Pais_y','Codigo'], axis = 1, inplace = True)

forbesglobal2000_2022

,Rank_nr,Empresa,Industria,Ingresos,Ganancias,Activos,Valor de Mercado,Margen de Rentabilidad,ROA,Año,Nombre de Pais_x,Nombre Continente,Codigo Continente,Codigo Pais
0,1,Berkshire Hathaway,Diversified Financials,276094.0,89795.0,958784.0,741476.0,0.325233,0.093655,2022,United States,North America,NaN,USA
1,4,JPMorgan Chase,Diversified Financials,124542.0,42115.0,3954687.0,374446.0,0.338159,0.010649,2022,United States,North America,NaN,USA
2,6,Amazon,Retailing,469822.0,33364.0,420549.0,1468398.0,0.071014,0.079334,2022,United States,North America,NaN,USA
3,7,Apple,Technology Hardware & Equipment,378697.0,100555.0,381191.0,2640316.0,0.265529,0.263792,2022,United States,North America,NaN,USA
4,9,Bank of America,Banking,96826.0,30995.0,3238223.0,303102.0,0.320110,0.009572,2022,United States,North America,NaN,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2232,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Europe,EU,SJM
2233,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oceania,OC,TKL
2234,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,North America,NaN,TCA
2235,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,North America,NaN,VIR


In [152]:
#Cambiamos el nombre de la columna Nombre de Pais_x
forbesglobal2000_2022.rename(columns={'Nombre de Pais_x':'Nombre Pais'}, inplace = True)

forbesglobal2000_2022

,Rank_nr,Empresa,Industria,Ingresos,Ganancias,...,Año,Nombre Pais,Nombre Continente,Codigo Continente,Codigo Pais
0,1,Berkshire Hathaway,Diversified Financials,276094.0,89795.0,...,2022,United States,North America,NaN,USA
1,4,JPMorgan Chase,Diversified Financials,124542.0,42115.0,...,2022,United States,North America,NaN,USA
2,6,Amazon,Retailing,469822.0,33364.0,...,2022,United States,North America,NaN,USA
3,7,Apple,Technology Hardware & Equipment,378697.0,100555.0,...,2022,United States,North America,NaN,USA
4,9,Bank of America,Banking,96826.0,30995.0,...,2022,United States,North America,NaN,USA
...,...,...,...,...,...,...,...,...,...,...,...
2232,-1,NaN,NaN,NaN,NaN,...,NaN,NaN,Europe,EU,SJM
2233,-1,NaN,NaN,NaN,NaN,...,NaN,NaN,Oceania,OC,TKL
2234,-1,NaN,NaN,NaN,NaN,...,NaN,NaN,North America,NaN,TCA
2235,-1,NaN,NaN,NaN,NaN,...,NaN,NaN,North America,NaN,VIR


In [153]:
forbesglobal2000_2022


,Rank_nr,Empresa,Industria,Ingresos,Ganancias,Activos,Valor de Mercado,Margen de Rentabilidad,ROA,Año,Nombre Pais,Nombre Continente,Codigo Continente,Codigo Pais
0,1,Berkshire Hathaway,Diversified Financials,276094.0,89795.0,958784.0,741476.0,0.325233,0.093655,2022,United States,North America,NaN,USA
1,4,JPMorgan Chase,Diversified Financials,124542.0,42115.0,3954687.0,374446.0,0.338159,0.010649,2022,United States,North America,NaN,USA
2,6,Amazon,Retailing,469822.0,33364.0,420549.0,1468398.0,0.071014,0.079334,2022,United States,North America,NaN,USA
3,7,Apple,Technology Hardware & Equipment,378697.0,100555.0,381191.0,2640316.0,0.265529,0.263792,2022,United States,North America,NaN,USA
4,9,Bank of America,Banking,96826.0,30995.0,3238223.0,303102.0,0.320110,0.009572,2022,United States,North America,NaN,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2232,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Europe,EU,SJM
2233,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Oceania,OC,TKL
2234,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,North America,NaN,TCA
2235,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,North America,NaN,VIR


In [54]:
forbesglobal2000_2022.head(10)

,Rank_nr,Empresa,Industria,Pais,Ingresos,Ganancias,Activos,Valor de Mercado,Margen de Rentabilidad,ROA,Año,Nombre de Pais_x,Codigo,Nombre Continente,Codigo Continente,Codigo Pais,Nombre de Pais_y
0,1,Berkshire Hathaway,Diversified Financials,United States,276094.0,89795.0,958784.0,741476.0,0.325233,0.093655,2022,United States,USA,North America,NaN,USA,United States of America
1,4,JPMorgan Chase,Diversified Financials,United States,124542.0,42115.0,3954687.0,374446.0,0.338159,0.010649,2022,United States,USA,North America,NaN,USA,United States of America
2,6,Amazon,Retailing,United States,469822.0,33364.0,420549.0,1468398.0,0.071014,0.079334,2022,United States,USA,North America,NaN,USA,United States of America
3,7,Apple,Technology Hardware & Equipment,United States,378697.0,100555.0,381191.0,2640316.0,0.265529,0.263792,2022,United States,USA,North America,NaN,USA,United States of America
4,9,Bank of America,Banking,United States,96826.0,30995.0,3238223.0,303102.0,0.320110,0.009572,2022,United States,USA,North America,NaN,USA,United States of America
5,11,Alphabet,IT Software & Services,United States,257488.0,76033.0,359268.0,1581723.0,0.295288,0.211633,2022,United States,USA,North America,NaN,USA,United States of America
6,12,Microsoft,IT Software & Services,United States,184903.0,71185.0,340389.0,2054367.0,0.384986,0.209128,2022,United States,USA,North America,NaN,USA,United States of America
7,15,ExxonMobil,Oil & Gas Operations,United States,280510.0,23040.0,338923.0,359731.0,0.082136,0.067980,2022,United States,USA,North America,NaN,USA,United States of America
8,18,Wells Fargo,Banking,United States,84123.0,20583.0,1939709.0,176767.0,0.244677,0.010611,2022,United States,USA,North America,NaN,USA,United States of America
9,19,Verizon Communications,Telecommunications Services,United States,134346.0,21520.0,366596.0,218022.0,0.160183,0.058702,2022,United States,USA,North America,NaN,USA,United States of America
